<a href="https://colab.research.google.com/github/vbanai/Linear_Logistic_Regression_Projects/blob/main/Tabular_playground_2021_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir - p ~/.kaggle 
! cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c tabular-playground-series-jan-2021

In [5]:
import os
os.environ['KAGGLE_USERNAME'] = "viktorbanai" # username from the json file
os.environ['KAGGLE_KEY'] = "9431db56af1a1a63cab88b4e516a2248" # key from the json file
!kaggle competitions download -c tabular-playground-series-jan-2021 # api copied from kaggle


 89% 34.0M/38.4M [00:00<00:00, 61.7MB/s]
100% 38.4M/38.4M [00:00<00:00, 78.2MB/s]
  0% 0.00/469k [00:00<?, ?B/s]
100% 469k/469k [00:00<00:00, 147MB/s]
 38% 9.00M/23.9M [00:00<00:00, 31.4MB/s]
100% 23.9M/23.9M [00:00<00:00, 60.4MB/s]


In [6]:
!unzip "/content/sample_submission.csv.zip" -d "/content/p/"

Archive:  /content/sample_submission.csv.zip
  inflating: /content/p/sample_submission.csv  


In [7]:
!unzip "/content/test.csv.zip" -d "/content/p/"
!unzip "/content/train.csv.zip" -d "/content/p/"

Archive:  /content/test.csv.zip
  inflating: /content/p/test.csv     
Archive:  /content/train.csv.zip
  inflating: /content/p/train.csv    


In [ ]:
dataframe_raw = pd.read_csv("/content/p/sample_submission.csv")
dataframe_raw.head()

In [ ]:
dataframe_raw = pd.read_csv("/content/p/test.csv")
dataframe_raw.head()


In [11]:
batch_size=32

dataframe_test = pd.read_csv("/content/p/test.csv")

ID=dataframe_test[['id']].to_numpy(dtype=str)

inputs_test_array=dataframe_test[['cont1', 'cont2','cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']].to_numpy(dtype=np.float32)
test_tensor = torch.from_numpy(inputs_test_array)
test_dl = DataLoader(test_tensor, batch_size)

In [12]:
dataframe_train00=pd.read_csv("/content/p/train.csv")
dataframe_train0 = dataframe_train00.copy(deep=True)

dataframe_train = dataframe_train0.sample(frac=1, random_state=8)

In [13]:
inputs_array=dataframe_train[['cont1', 'cont2','cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']].to_numpy(dtype=np.float32)
targets_array=dataframe_train[['target']].to_numpy(dtype=np.float32)

inputs_tensor = torch.from_numpy(inputs_array)
targets_tensor= torch.from_numpy(targets_array)

dataset = TensorDataset(inputs_tensor, targets_tensor)

train_ds, val_ds =train_test_split(dataset, test_size=0.15, random_state=42)






In [14]:



train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, num_workers=2, pin_memory=True)

input_size=14
output_size=1

In [15]:
class Tabularplayground(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)             
        
    def forward(self, xb):
        out = self.linear(xb)                          
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        out = self(inputs)         
        loss = F.mse_loss(out, targets)                         
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.mse_loss(out, targets) 
        acc = accuracy(out, targets)                      
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        #epoch_acc = torch.stack(batch_accs).mean()
        epoch_acc = sum(batch_accs)/len(batch_accs)
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 2 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch+1, result['val_loss'], result['val_acc']))

def accuracy(outputs, labels):
    counter=0
    for i in range(len(labels)):
      if outputs[i]>labels[i]*0.8 and outputs[i]<labels[i]*1.2:
        counter +=1
    return counter/len(labels)

    #return torch.tensor(torch.sum(outputs == labels).item() / len(outputs))
    #return torch.tensor(torch.sum(outputs > labels*0.8 and outputs<labels*1.2).item() / len(outputs))


model = Tabularplayground()

In [16]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [17]:
device = get_default_device()
device

device(type='cuda')

In [18]:
model = to_device(Tabularplayground(), device)

In [19]:


def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        train_losses = []
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [20]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
test_dl=DeviceDataLoader(test_dl, device)
to_device(model, device)

Tabularplayground(
  (linear): Linear(in_features=14, out_features=1, bias=True)
)

In [21]:
num_epochs = 50
#opt_func = torch.optim.Adam
lr = 0.0001

history = fit(num_epochs, lr, model, train_dl, val_dl)

Epoch [2], val_loss: 1.5591, val_acc: 0.7919
Epoch [4], val_loss: 1.2065, val_acc: 0.8473
Epoch [6], val_loss: 1.0408, val_acc: 0.8762
Epoch [8], val_loss: 0.9482, val_acc: 0.8943
Epoch [10], val_loss: 0.8865, val_acc: 0.9065
Epoch [12], val_loss: 0.8403, val_acc: 0.9159
Epoch [14], val_loss: 0.8028, val_acc: 0.9226
Epoch [16], val_loss: 0.7716, val_acc: 0.9291
Epoch [18], val_loss: 0.7448, val_acc: 0.9346
Epoch [20], val_loss: 0.7218, val_acc: 0.9393
Epoch [22], val_loss: 0.7016, val_acc: 0.9429
Epoch [24], val_loss: 0.6842, val_acc: 0.9468
Epoch [26], val_loss: 0.6686, val_acc: 0.9480
Epoch [28], val_loss: 0.6551, val_acc: 0.9508
Epoch [30], val_loss: 0.6430, val_acc: 0.9517
Epoch [32], val_loss: 0.6324, val_acc: 0.9542
Epoch [34], val_loss: 0.6228, val_acc: 0.9559
Epoch [36], val_loss: 0.6143, val_acc: 0.9573
Epoch [38], val_loss: 0.6066, val_acc: 0.9584
Epoch [40], val_loss: 0.5997, val_acc: 0.9592
Epoch [42], val_loss: 0.5935, val_acc: 0.9605
Epoch [44], val_loss: 0.5879, val_acc:

In [ ]:
test_result=[]

for batch in test_dl:
  outputs_test = model(batch)
  test_result.append(outputs_test)

print(test_result[0])
 

In [23]:
 test_numpy=[]
 for i in test_result:
   i2=i.cpu().detach().numpy()
   test_numpy.append(i2)

test_numpy_array=np.concatenate(test_numpy)




In [ ]:
id=ID.reshape(1,200000)
Test_Numpy_Array=test_numpy_array.reshape(1,200000)



arr = np.concatenate((id, Test_Numpy_Array), axis=0)


print("id", "     ", "target")

for i in range(len(arr[0])):
  print(arr[0][i], " ", arr[1][i])



In [35]:
import csv

with open("mycsv", 'w') as f:
  thewriter=csv.writer(f)

  thewriter.writerow(['id', 'target'])
  for i in range(len(arr[0])):
    thewriter.writerow([arr[0][i], arr[1][i]])




In [ ]:
finaldoc = pd.read_csv("/content/mycsv")
finaldoc.head()

In [ ]:
from google.colab import files
files.download('mycsv')